In [1]:
# ---------------- Cell 1: Imports & defaults ----------------
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks, optimizers
import os

# sensible defaults (override earlier cells if already defined)
IMG_SIZE = globals().get("IMG_SIZE", (224, 224))   # model input size
BATCH_SIZE = globals().get("BATCH_SIZE", 32)
SEED = globals().get("SEED", 42)
NUM_CLASSES = 10

AUTOTUNE = tf.data.AUTOTUNE

print("IMG_SIZE:", IMG_SIZE, "BATCH_SIZE:", BATCH_SIZE, "SEED:", SEED)


IMG_SIZE: (224, 224) BATCH_SIZE: 32 SEED: 42


In [2]:
# ---------------- Cell 2: Load CIFAR-10 and create train/val/test tf.data pipelines ----------------
(x_train_all, y_train_all), (x_test_all, y_test_all) = tf.keras.datasets.cifar10.load_data()
y_train_all = y_train_all.flatten()
y_test_all = y_test_all.flatten()

# Stratified split: 90% train, 10% validation from training set
X_tr, X_val, y_tr, y_val = train_test_split(
    x_train_all, y_train_all, test_size=0.1, random_state=SEED, stratify=y_train_all
)

print("Shapes -> X_tr:", X_tr.shape, "X_val:", X_val.shape, "X_test:", x_test_all.shape)


Shapes -> X_tr: (45000, 32, 32, 3) X_val: (5000, 32, 32, 3) X_test: (10000, 32, 32, 3)


In [3]:
# ---------------- Cell 3: Preprocessing and tf.data datasets ----------------
def preprocess_train(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, IMG_SIZE)  # lazy resize per example
    # Augmentations:
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.08)
    image.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

def preprocess_val(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, IMG_SIZE)
    image.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

# Create tf.data train & val pipelines
train_ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
train_ds = train_ds.shuffle(buffer_size=5000, seed=SEED)
train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(preprocess_val, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Also create a test dataset (no augmentation)
test_ds = tf.data.Dataset.from_tensor_slices((x_test_all, y_test_all))
def preprocess_test(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, IMG_SIZE)
    image.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label
test_ds = test_ds.map(preprocess_test, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

print("train_ds element spec:", train_ds.element_spec)
print("val_ds   element spec:", val_ds.element_spec)
print("test_ds  element spec:", test_ds.element_spec)


train_ds element spec: (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))
val_ds   element spec: (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))
test_ds  element spec: (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))


In [4]:
# ---------------- Cell 4: Build or reuse 'model' (transfer learning) ----------------
# If a model already exists in the notebook (e.g., you defined 'model' earlier), we will keep it.
# Otherwise build a lightweight transfer learning model using MobileNetV2 backbone.

if 'model' in globals():
    print("Using existing model object found in workspace.")
else:
    # Build model with pretrained mobile backbone (not include top)
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )
    base.trainable = False  # freeze backbone for head training

    inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    print("Built new MobileNetV2-based model (backbone frozen).")

# Compile
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Built new MobileNetV2-based model (backbone frozen).


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,588,490 (9.87 MB)

 Trainable params: 330,506 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# ---------------- Cell 5: Callbacks ----------------
checkpoint_path = "best_model_checkpoint.keras"
cb_list = [
    callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss', mode='min'),
    callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]


In [ ]:
# ---------------- Cell 6: Train classifier head (backbone frozen) ----------------
EPOCHS_HEAD = 4  # tune as needed

# ensure backbone is frozen for head training
try:
    base.trainable = False
except NameError:
    pass  # if base isn't defined because model was prebuilt differently

history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD,
    callbacks=cb_list,
    verbose=2
)


Epoch 1/4


In [ ]:
# ---------------- Cell 7: Optional fine-tune (unfreeze some backbone layers) ----------------
FINE_TUNE = True
UNFREEZE_AT = -50  # number of layers from end to unfreeze (tweak as needed)

if FINE_TUNE:
    # Unfreeze from the last UNFREEZE_AT layers
    try:
        base.trainable = True
        # Freeze earlier layers
        for layer in base.layers[:UNFREEZE_AT]:
            layer.trainable = False
        for layer in base.layers[UNFREEZE_AT:]:
            layer.trainable = True

        model.compile(
            optimizer=optimizers.Adam(learning_rate=1e-4),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        EPOCHS_FINE = 6
        history_fine = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS_FINE,
            callbacks=cb_list,
            verbose=2
        )
    except NameError:
        print("Base model not available for fine-tuning; skip.")


In [ ]:
# ---------------- Cell 8: Evaluate using val_ds (same preprocessing as training) ----------------
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}")

test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")


In [ ]:
# ---------------- Cell 9: Save model and prediction helper ----------------
# Save as TensorFlow SavedModel format
model_save_path = "final_model_saved"
model.save(model_save_path, save_format="tf")
print("Saved model to", model_save_path)

# Single-image prediction helper
import numpy as np

def predict_single_image(img_array, model, class_names=None):
    """
    img_array : numpy array, shape (H,W,3) in uint8 or float [0,255]
    model : the trained tf.keras Model
    class_names: optional list mapping label index to human name
    """
    img = tf.cast(img_array, tf.float32) / 255.0
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.expand_dims(img, axis=0)  # batch dim
    probs = model.predict(img)
    pred_idx = int(np.argmax(probs, axis=-1)[0])
    confidence = float(np.max(probs))
    label = class_names[pred_idx] if class_names is not None else pred_idx
    return label, confidence, probs[0]

# class names for CIFAR-10:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


In [ ]:
# ---------------- Cell 10: Demo prediction on first test image ----------------
sample_img = x_test_all[0]          # original CIFAR test image (32x32)
label, conf, probs = predict_single_image(sample_img, model, class_names)
print("Predicted:", label, "Confidence:", conf)
print("Top probs:", np.round(probs, 4))


In [ ]:
# 🧠 PART 1: What this experiment is about (in simple words)
# This experiment shows how to perform Object Detection / Image Classification using Transfer Learning —
# a method where we reuse a pre-trained CNN (like VGG16, ResNet, or MobileNet) that has already learned features from a large dataset (e.g., ImageNet).
# Instead of training from scratch, we:
# 1.	Load a pre-trained CNN model
# 2.	Freeze its early layers (they already know how to detect general features like edges, shapes)
# 3.	Add new classifier layers on top for our specific dataset
# 4.	Train only the new layers
# 5.	Optionally unfreeze deeper layers and fine-tune them for better accuracy.
# 🧩 Think of it like taking a student who already knows how to recognize shapes and teaching them to recognize a new type of object faster — instead of starting from zero.
# ________________________________________
# 📘 PART 2: Exam-Ready Notes
# 🧩 Title
# Object Detection using Transfer Learning of CNN Architectures
# ________________________________________
# 🎯 Aim
# To build an object detection/classification model using transfer learning with a pre-trained CNN,
# by freezing early layers and training new classifier layers for the given dataset.
# ________________________________________
# 🧠 Theory (in simple words)
# •	CNN (Convolutional Neural Network) models like VGG16, ResNet50, InceptionV3, and MobileNet are trained on massive datasets like ImageNet with millions of images.
# •	These networks learn to detect low-level features (edges, shapes, colors) and high-level features (objects, textures).
# •	Transfer Learning uses these pre-learned features and adapts them to a smaller, specific dataset — saving time and improving performance.
# •	Steps:
# 1.	Load a pre-trained CNN (e.g., VGG16)
# 2.	Freeze the convolutional base layers
# 3.	Add new Dense layers for classification
# 4.	Train only these new layers
# 5.	Optionally unfreeze some deeper layers and fine-tune for accuracy
# 📊 Result
# •	The transfer learning model successfully classifies images with high accuracy (usually >90% for small datasets).
# •	Training time is much shorter compared to building a CNN from scratch.
# •	Model learns both general (edges, textures) and task-specific features.
# ________________________________________
# ✅ Conclusion
# •	Transfer Learning reuses pre-trained CNN knowledge to efficiently solve new tasks.
# •	Freezing lower layers preserves general visual features.
# •	Fine-tuning deeper layers improves model accuracy.
# •	This method is widely used in modern computer vision (e.g., image classification, object detection, medical imaging).
# ________________________________________
# 💬 PART 3: Viva Questions (Simple Answers)
# Question	Answer (Simple)
# What is Transfer Learning?	Using a model pre-trained on one dataset for another similar task.
# Why freeze the base layers?	To keep general visual features learned from large datasets.
# Why add new layers?	To train on new, task-specific classes.
# What dataset is the base model pre-trained on?	ImageNet (commonly used, with 1,000 classes).
# What optimizer is used?	Adam or SGD.
# What happens if you unfreeze too many layers?	The model may overfit or forget earlier knowledge (catastrophic forgetting).
# What is fine-tuning?	Adjusting a few layers of a pre-trained model with a low learning rate.
# Why use dropout?	To prevent overfitting during training.
# What are examples of pre-trained models?	VGG16, ResNet50, MobileNet, InceptionV3, EfficientNet.
